In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
24,270,20297127,카페오작교,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135953,6070.0,NaN,1,NaN,127.049293,37.522951,7호선 청담,398.760647,NaN
25,282,23238698,들꽃누리,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,150070,7412.0,NaN,1,NaN,126.898633,37.497079,7호선 대림(구로구청),368.888504,NaN
34,469,20017448,커피빈,코리아청담에스점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135955,6011.0,NaN,1,109,127.047216,37.525250,7호선 청담,675.120538,NaN
37,497,24327928,건대포토카페프린세스,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143190,5073.0,NaN,1,NaN,127.068745,37.539802,7호선 건대입구,60.940672,NaN
47,586,20023467,커피빈,코리아청담성당점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135953,6074.0,NaN,2,NaN,127.050021,37.519902,7호선 청담,229.848907,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12149,135223,17254892,안다미로팬케이크,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143915,5012.0,NaN,NaN,NaN,127.071012,37.544992,7호선 어린이대공원(세종대),355.054760,NaN
12150,135226,17252483,초코라떼,건대역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,143854,5066.0,NaN,7,NaN,127.074625,37.538335,7호선 어린이대공원(세종대),592.981894,NaN
12152,135251,17207986,카페,동네항동점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,152140,8362.0,NaN,1,NaN,126.821130,37.482852,7호선 온수(성공회대입구),567.166627,NaN
12159,135345,17243108,코끼리키즈까페,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,152815,8352.0,NaN,2,NaN,126.854261,37.486412,7호선 광명사거리,478.139096,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
24,카페오작교,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로92길 26,7호선 청담,398.760647,NaN,청담,청담역 카페오작교
25,들꽃누리,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 대림로 183,7호선 대림(구로구청),368.888504,NaN,대림,대림역 들꽃누리
34,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 압구정로 461,7호선 청담,675.120538,NaN,청담,청담역 커피빈
37,건대포토카페프린세스,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 동일로20길 98,7호선 건대입구,60.940672,NaN,건대입구,건대입구역 건대포토카페프린세스
47,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 삼성로 716,7호선 청담,229.848907,NaN,청담,청담역 커피빈
...,...,...,...,...,...,...,...,...,...
12149,안다미로팬케이크,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 군자로 19,7호선 어린이대공원(세종대),355.054760,NaN,어린이대공원,어린이대공원역 안다미로팬케이크
12150,초코라떼,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 아차산로36길 5,7호선 어린이대공원(세종대),592.981894,NaN,어린이대공원,어린이대공원역 초코라떼
12152,카페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 연동로11길 19,7호선 온수(성공회대입구),567.166627,NaN,온수,온수역 카페
12159,코끼리키즈까페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 개봉로3길 36,7호선 광명사거리,478.139096,NaN,광명사거리,광명사거리역 코끼리키즈까페


In [4]:
items = data["검색어"][104:196]
items

816          남구로역 스타벅스
825        어린이대공원역 카페두
833            대림역 씨카페
834           노원역 카페칼디
856      태릉입구역 갤러리힐링카페
             ...      
1404           사가정역 카페
1405           사가정역 카페
1422    총신대입구역 신묘한사주타로
1428       어린이대공원역 엔조이
1432       뚝섬유원지역 쥬씨차얌
Name: 검색어, Length: 92, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/92
식당 존재
https://place.map.kakao.com/7961927
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/92
식당 존재
https://place.map.kakao.com/22674971
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/92
식당 존재
https://place.map.kakao.com/27235482
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/92
식당 존재 x
진행상황 : 5/92
식당 존재
https://place.map.kakao.com/175884252
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/92
식당 존재
https://place.map.kakao.com/26475260
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/92
식당 존재 x
진행상황 : 8/92
식당 존재
https://place.map.kakao.com/1542009728
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/92
식당 존재
https://place.map.kakao.com/11435932
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/92
식당 존재
https://place.map.kakao.com/22575303
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/92
식당 존재
https://place.map.kakao.com/8403571
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/92
식당 존재
https://place.map.kakao.com/1652260621
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/92
식당 존재 x
진행상황 : 14/92
식당 존재
https://place.map.kakao.com/27588641
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/92
식당 존재 

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,ljr96p,남구로역 스타벅스,5점,2019.03.31.
1,e505t7,남구로역 스타벅스,5점,2017.09.28.
2,beie1s,남구로역 스타벅스,4점,2017.08.07.
3,None,남구로역 스타벅스,None,2017.01.16.
4,None,남구로역 스타벅스,None,2016.10.11.
...,...,...,...,...
413,1bflvv2,사가정역 카페,5점,2020.09.10.
414,ed7n2u,사가정역 카페,5점,2020.08.16.
415,gf37t7,사가정역 카페,1점,2019.07.11.
416,vneom4,어린이대공원역 엔조이,1점,2021.04.02.


In [8]:
img_elem

,ItemID,img_url
0,남구로역 스타벅스,None
1,남구로역 스타벅스,None
2,남구로역 스타벅스,None
3,남구로역 스타벅스,None
4,남구로역 스타벅스,None
...,...,...
413,사가정역 카페,None
414,사가정역 카페,None
415,사가정역 카페,None
416,어린이대공원역 엔조이,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
24,카페오작교,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 도산대로92길 26,7호선 청담,398.760647,https://place.map.kakao.com/7961927,청담,청담역 카페오작교
25,들꽃누리,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 대림로 183,7호선 대림(구로구청),368.888504,https://place.map.kakao.com/22674971,대림,대림역 들꽃누리
34,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 압구정로 461,7호선 청담,675.120538,https://place.map.kakao.com/27235482,청담,청담역 커피빈
37,건대포토카페프린세스,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 동일로20길 98,7호선 건대입구,60.940672,None,건대입구,건대입구역 건대포토카페프린세스
47,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 삼성로 716,7호선 청담,229.848907,https://place.map.kakao.com/175884252,청담,청담역 커피빈
...,...,...,...,...,...,...,...,...,...
12149,안다미로팬케이크,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 군자로 19,7호선 어린이대공원(세종대),355.054760,NaN,어린이대공원,어린이대공원역 안다미로팬케이크
12150,초코라떼,커피점/카페,커피전문점/카페/다방,서울특별시 광진구 아차산로36길 5,7호선 어린이대공원(세종대),592.981894,NaN,어린이대공원,어린이대공원역 초코라떼
12152,카페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 연동로11길 19,7호선 온수(성공회대입구),567.166627,NaN,온수,온수역 카페
12159,코끼리키즈까페,커피점/카페,커피전문점/카페/다방,서울특별시 구로구 개봉로3길 36,7호선 광명사거리,478.139096,NaN,광명사거리,광명사거리역 코끼리키즈까페


In [10]:
img_elem

,ItemID,img_url
0,남구로역 스타벅스,None
1,남구로역 스타벅스,None
2,남구로역 스타벅스,None
3,남구로역 스타벅스,None
4,남구로역 스타벅스,None
...,...,...
413,사가정역 카페,None
414,사가정역 카페,None
415,사가정역 카페,None
416,어린이대공원역 엔조이,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,ljr96p,남구로역 스타벅스,5점,2019.03.31.
1,e505t7,남구로역 스타벅스,5점,2017.09.28.
2,beie1s,남구로역 스타벅스,4점,2017.08.07.
3,None,남구로역 스타벅스,None,2017.01.16.
4,None,남구로역 스타벅스,None,2016.10.11.
...,...,...,...,...
413,1bflvv2,사가정역 카페,5점,2020.09.10.
414,ed7n2u,사가정역 카페,5점,2020.08.16.
415,gf37t7,사가정역 카페,1점,2019.07.11.
416,vneom4,어린이대공원역 엔조이,1점,2021.04.02.


In [17]:
review_elem

,ItemID,review
0,남구로역 스타벅스,친절합니다. 매장이 좁다고 마음도 좁지 않았습니다.
1,남구로역 스타벅스,깔끔하고 고객응대가 친절함
2,남구로역 스타벅스,좋아요
3,남구로역 스타벅스,좋아요
4,남구로역 스타벅스,좋아요
...,...,...
413,사가정역 카페,분위기도 좋은데 커피까지 너무 맛있어요. 딱 하나 아쉬웠던 점이 실내에서 마스크 안...
414,사가정역 카페,커피가 고소하고 달달해요
415,사가정역 카페,12:15 왔는데 문 잠겨있음 전화하고 가시길
416,어린이대공원역 엔조이,돈 넣은 금액보다 적게 충전이 되서..점주님에게 연락드렸는데 일주일 넘게 연락안옴....


In [13]:
img_elem.head()

,ItemID,img_url
0,남구로역 스타벅스,None
1,남구로역 스타벅스,None
2,남구로역 스타벅스,None
3,남구로역 스타벅스,None
4,남구로역 스타벅스,None


In [18]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




     UserID       ItemID Rating    Timestamp
416  vneom4  어린이대공원역 엔조이     1점  2021.04.02.
417  8o1bqq  어린이대공원역 엔조이     3점  2020.03.13.
          ItemID img_url
416  어린이대공원역 엔조이    None
417  어린이대공원역 엔조이    None
          ItemID                                             review
416  어린이대공원역 엔조이  돈 넣은 금액보다 적게 충전이 되서..점주님에게 연락드렸는데 일주일 넘게 연락안옴....
417  어린이대공원역 엔조이  위치도 괜찮고 한데 그렇게 저렴한거같진않고 이불빨래했는데 5000원 /건조기 100...


In [20]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe(final_2).csv')
rating_df

,UserID,ItemID,Rating,Timestamp
0,ljr96p,남구로역 스타벅스,5점,2019.03.31.
1,e505t7,남구로역 스타벅스,5점,2017.09.28.
2,beie1s,남구로역 스타벅스,4점,2017.08.07.
3,None,남구로역 스타벅스,None,2017.01.16.
4,None,남구로역 스타벅스,None,2016.10.11.
...,...,...,...,...
413,1bflvv2,사가정역 카페,5점,2020.09.10.
414,ed7n2u,사가정역 카페,5점,2020.08.16.
415,gf37t7,사가정역 카페,1점,2019.07.11.
416,vneom4,어린이대공원역 엔조이,1점,2021.04.02.


In [21]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe_User(final_2).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_cafe_img(final_2).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline7_cafe_review(final_2).csv')

In [22]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


49

In [23]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_final_2url.csv')